In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pathlib

# Train a model

In [2]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = (train_images / 255.0).astype(np.float32)
test_images = (test_images / 255.0).astype(np.float32)

# Define the model architecture
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(
  train_images,
  train_labels,
  epochs=5,
  validation_data=(test_images, test_labels)
)

11501568/11490434 [==============================] - 1s 0us/step


2022-07-02 05:22:48.304245: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-07-02 05:22:48.639325: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2951 - accuracy: 0.9166 - val_loss: 0.1398 - val_accuracy: 0.9602
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1163 - accuracy: 0.9667 - val_loss: 0.0867 - val_accuracy: 0.9740
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0836 - accuracy: 0.9757 - val_loss: 0.0725 - val_accuracy: 0.9764
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0689 - accuracy: 0.9796 - val_loss: 0.0685 - val_accuracy: 0.9781
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0596 - accuracy: 0.9826 - val_loss: 0.0572 - val_accuracy: 0.9809


In [3]:
models_dir = pathlib.Path('models')
models_dir.mkdir(exist_ok=True, parents=True)
model.save(f'{models_dir}/tf_model.h5')

# Various Quantization Techniques

**Convert to a TensorFlow Lite model**

TensorFlow Lite converts weights to 8 bit precision as part of model conversion from tensorflow graphdefs to TensorFlow Lite's flat buffer format.

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_model_file = models_dir/"tflite_model.tflite"
tflite_model_file.write_bytes(tflite_model)

2022-07-02 05:24:12.013618: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-07-02 05:24:12.712141: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-07-02 05:24:12.712472: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-07-02 05:24:12.718218: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.015ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-07-02 05:24:12.785490: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2022-07-02 05:24:12.785591: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:

84500

**Dynamic Quantization**

Dynamic range quantization achieves a 4x reduction in the model size.
The weights are quantized post training and the activations are quantized dynamically at inference in this method.

In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_dynamic_quant_model = converter.convert()
tflite_dynamic_quant_model_file = models_dir/"tflite_dynamic_quant_model.tflite"
tflite_dynamic_quant_model_file.write_bytes(tflite_dynamic_quant_model)

2022-07-02 05:24:14.151124: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-07-02 05:24:14.151344: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-07-02 05:24:14.152997: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.009ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-07-02 05:24:14.193063: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2022-07-02 05:24:14.193140: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.
2022-07-02 05:24:14.214368: I tensorflow/lite/tools/optimize/quantize_weights.cc:225] Skipping quantization of tensor sequential/conv2d/Conv2

23904

**Integer Quantization**

Integer quantization is an optimization strategy that converts 32-bit floating-point numbers (such as weights and activation outputs) to the nearest 8-bit fixed-point numbers. This results in a smaller model and increased inferencing speed.

To quantize the variable data (such as model input/output and intermediates between layers), you need to provide a RepresentativeDataset. This is a generator function that provides a set of input data that's large enough to represent typical values. It allows the converter to estimate a dynamic range for all the variable data. (The dataset does not need to be unique compared to the training or evaluation dataset.) To support multiple inputs, each representative data point is a list and elements in the list are fed to the model according to their indices.

In [6]:
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_int_quant_model = converter.convert()
tflite_int_quant_model_file = models_dir/"tflite_int_quant_model.tflite"
tflite_int_quant_model_file.write_bytes(tflite_dynamic_quant_model)

2022-07-02 05:24:15.276001: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-07-02 05:24:15.276232: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-07-02 05:24:15.278271: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.015ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-07-02 05:24:15.322859: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2022-07-02 05:24:15.322950: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.
fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 3


23904

**Float 16 Quantization**

Converting weights to 16-bit floating point values during model conversion from TensorFlow to TensorFlow Lite's flat buffer format, results in a 2x reduction in model size. Some hardware, like GPUs, can compute natively in this reduced precision arithmetic, realizing a speedup over traditional floating point execution. The Tensorflow Lite GPU delegate can be configured to run in this way. However, a model converted to float16 weights can still run on the CPU without additional modification: the float16 weights are upsampled to float32 prior to the first inference. This permits a significant reduction in model size in exchange for a minimal impacts to latency and accuracy.

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_float16_quant_model = converter.convert()
tflite_float16_quant_model_file = models_dir/"tflite_float16_quant_model.tflite"
tflite_float16_quant_model_file.write_bytes(tflite_float16_quant_model)

2022-07-02 05:24:17.091118: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-07-02 05:24:17.091313: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-07-02 05:24:17.093040: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.008ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-07-02 05:24:17.135349: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2022-07-02 05:24:17.135410: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.


44432

**16x8 Quantization**

Converting activations to 16-bit integer values and weights to 8-bit integer values during model conversion from TensorFlow to TensorFlow Lite's flat buffer format can improve accuracy of the quantized model significantly, when activations are sensitive to the quantization, while still achieving almost 3-4x reduction in model size. Moreover, this fully quantized model can be consumed by integer-only hardware accelerators.

In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]
tflite_16x8_quant_model = converter.convert()
tflite_16x8_quant_model_file = models_dir/"tflite_16x8_quant_model.tflite"
tflite_16x8_quant_model_file.write_bytes(tflite_16x8_quant_model)

2022-07-02 05:24:18.381922: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-07-02 05:24:18.382130: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-07-02 05:24:18.383737: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.007ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-07-02 05:24:18.426805: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2022-07-02 05:24:18.426879: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.
2022-07-02 05:24:18.449787: I tensorflow/lite/tools/optimize/quantize_weights.cc:225] Skipping quantization of tensor sequential/conv2d/Conv2

23904

# Comparison and Evaluation

In [9]:
ls -la models/

total 476
drwxr-xr-x 2 root root   4096 Jul  2 05:24 ./
drwxr-xr-x 3 root root   4096 Jul  2 05:24 ../
-rw-r--r-- 1 root root 271416 Jul  2 05:24 tf_model.h5
-rw-r--r-- 1 root root  23904 Jul  2 05:24 tflite_16x8_quant_model.tflite
-rw-r--r-- 1 root root  23904 Jul  2 05:24 tflite_dynamic_quant_model.tflite
-rw-r--r-- 1 root root  44432 Jul  2 05:24 tflite_float16_quant_model.tflite
-rw-r--r-- 1 root root  23904 Jul  2 05:24 tflite_int_quant_model.tflite
-rw-r--r-- 1 root root  84500 Jul  2 05:24 tflite_model.tflite


In [10]:
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    # Run predictions on every image in the "test" dataset.
    prediction_digits = []
    for test_image in test_images:
        # Pre-processing: add batch dimension and convert to float32 to match with
        # the model's input data format.
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)

        # Run inference.
        interpreter.invoke()

        # Post-processing: remove batch dimension and find the digit with highest
        # probability.
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)

    # Compare prediction results with ground truth labels to calculate accuracy.
    accurate_count = 0
    for index in range(len(prediction_digits)):
        if prediction_digits[index] == test_labels[index]:
            accurate_count += 1
    accuracy = accurate_count * 1.0 / len(prediction_digits)

    return accuracy

In [11]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
evaluate_model(interpreter)

0.9809

In [12]:
interpreter_dynamic_quant = tf.lite.Interpreter(model_path=str(tflite_dynamic_quant_model_file))
interpreter_dynamic_quant.allocate_tensors()
evaluate_model(interpreter_dynamic_quant)

0.9803

In [13]:
interpreter_int_quant = tf.lite.Interpreter(model_path=str(tflite_int_quant_model_file))
interpreter_int_quant.allocate_tensors()
evaluate_model(interpreter_int_quant)

0.9803

In [14]:
interpreter_float16_quant = tf.lite.Interpreter(model_path=str(tflite_float16_quant_model_file))
interpreter_float16_quant.allocate_tensors()
evaluate_model(interpreter_float16_quant)

0.9808

In [15]:
interpreter_16x8_quant = tf.lite.Interpreter(model_path=str(tflite_16x8_quant_model_file))
interpreter_16x8_quant.allocate_tensors()
evaluate_model(interpreter_16x8_quant)

0.9803